In [ ]:
from nltk.corpus import wordnet as wn
import json
from itertools import chain
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('words')
!pip install language-tool-python
import language_tool_python
import re
import spacy
nlp = spacy.load("en_core_web_sm")
!pip install --upgrade gingerit
import gingerit




! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error

# Sentence PreProcessing

**`is_proper_english(sentence)`**: This function checks if a given sentence is considered proper English. It tokenizes the sentence into individual words, tags each word with its part of speech using NLTK, and then verifies if all words are recognized as English words with valid parts of speech. The function returns `True` if the sentence is considered proper English, and `False` otherwise.

**`check_english_sentence(sentence)`**: This function checks if a sentence is written in English. It removes non-alphabetic characters, converts the sentence to lowercase, and uses the `nlp` object from SpaCy to process the sentence. It then verifies that each token in the processed sentence is composed of alphabetic ASCII characters and has the language attribute set to "en" (English). The function returns `True` if all tokens in the sentence are determined to be English, and `False` otherwise.

**`remove_symbols(sentence)`**: This function removes symbols and special characters from a given sentence while retaining parentheses, semicolons, exclamation marks, and quotation marks. It uses regular expressions to perform the removal and returns the modified sentence.

**`correct_sentence(sentence)`**: This function utilizes the LanguageTool library to correct grammatical errors in a given sentence. It takes the sentence as input, applies language-based rules to identify and correct errors, and returns the corrected sentence.

By using these functions, you can ensure the proper validation, cleaning, and correction of sentences in your language processing tasks.

In [ ]:
def is_proper_english(sentence):
    # Download the necessary resources for the NLTK library
    
    
    # Tokenize the sentence into individual words
    words = nltk.word_tokenize(sentence)
    
    # Tag each word with its part of speech
    tagged_words = nltk.pos_tag(words)
    
    # Check if all words are recognized as English words and their corresponding part of speech is valid
    for word, tag in tagged_words:
        if not word.isalpha() or tag not in ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            return False
    
    return True

# def check_english_sentence(sentence):
#     sentence = re.sub(r'[^\w\s]', ' ', sentence)

#     # Remove extra whitespace and convert to lowercase
#     sentence = re.sub(r'\s+', ' ', sentence).strip().lower()
#     words = nltk.word_tokenize(sentence)
#     print(words)
#     english_vocab = set(w.lower() for w in nltk.corpus.words.words())
#     return all(word.lower() in english_vocab for word in words)

def check_english_sentence(sentence):
    sentence = re.sub(r'[^\w\s]', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip().lower()
    doc = nlp(sentence)

    return all(token.is_alpha and token.is_ascii and token.lang_ == "en" for token in doc)
def remove_symbols(sentence):
    # remove mathematical symbols
    sentence = re.sub(r'[+\-*\/=<>]', '', sentence)

    # remove currency symbols
#     sentence = re.sub(r'[£€¥$]', '', sentence)

    # remove special characters except for parentheses, semicolons, exclamation marks, and quotation marks
    sentence = re.sub(r'[^\w\s();!"\']', '', sentence)

    return sentence

def correct_sentence(sentence):
    tool = language_tool_python.LanguageTool('en-US')
    corrected = tool.correct(sentence)
    return corrected



# Getting all jsons line by line from wiktionary English Data


In [ ]:
# Getting all jsons line by line from wiktionary English Data



lst = []
with open("/kaggle/input/wiktionarywebster/kaikki.org-dictionary-English.json", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        lst.append(data)

# Getting the list of credible words from NLTK & Webster

In [ ]:
# Getting all words that we have in NLTK & Webster
allWords = set(chain(*[ss.lemma_names() for ss in wn.all_synsets()]))
print(len(allWords))
with open("/kaggle/input/wiktionarywebster/dictionary_compact.json",'r') as f:
        data = json.load(f)
for i in data:
        allWords.add(i)        
len(allWords)

# Function which will link every json to the word it belongs
The `formatUsage(body)` function is responsible for processing the data and extracting a JSON structure containing the definitions and examples of a word. It iterates over the senses in the provided `body` and extracts the first gloss (definition) from each sense. If an example is available, it removes symbols from the example text and filters out non-English examples. The resulting data is then appended to the `usageList` list, which contains objects representing each definition with its corresponding examples.

The `formatAudio(body)` function processes the data and extracts a JSON structure containing audio links for the word. It checks if the provided `body` contains sounds, and if so, it iterates over the sounds and extracts the `mp3_url` and `tags` (if available) for each sound. The extracted audio data is appended to the `audioList` list, ensuring that duplicate tags are not included.

The `formatIPA(body)` function extracts the International Phonetic Alphabet (IPA) representation of the word from the provided `body`. It checks if the `body` contains sounds and if any of the sounds have an `ipa` attribute. If found, the IPA representation is stored in the `ipa` variable.

The `formatObject(body)` function utilizes the aforementioned functions and processes the provided `body` to create a formatted JSON object representing the word. It extracts the part of speech, etymology text and number (if available), definitions with examples, audio links, IPA representation, and the source (Wiktionary) for the word. The resulting JSON object is returned.

By utilizing these functions, you can preprocess the data and extract relevant information from the webster source, ensuring a structured representation of the word's definitions, examples, audio links, IPA, and other details from Wiktionary.

In [ ]:
def formatUsage(body):
    usageList = []
    definitions = []
    try:
        
        for i in body['senses']:
            definition = i['glosses'][0].split('\n') if 'glosses' in i else [""]
            definition = definition[0]
            if definition in definitions:
                continue
            definitions.append(definition)

            if 'examples' in i:
                for j in i['examples']:
                    j['source'] = 'Wiktionary'
                    j['text'] = remove_symbols(j['text'])

            examples = i['examples'] if 'examples' in i else []
            filtered_data = list(filter(lambda x: check_english_sentence(x['text']) , examples))

            usageList.append({
                'definition': {
                                'gloss':definition,
                                'source':'Wiktionary',
                },
                'examples': filtered_data
            })
    except:
        return usageList
    
    return usageList

def formatAudio(body):
    audioList = []
    if 'sounds' in body:
        
        for j in body['sounds']:
            if ('mp3_url' in j):
                for i in audioList:
                    if 'tags' in j and  i['tags'] == j['tags'][0]:
                        continue
                
                audioList.append({
                    'audioLink': j['mp3_url'],
                    'tags' : j['tags'][0] if 'tags' in j else "",
                    'source': 'Wiktionary',
                    

                })
                
    return audioList

def formatIPA(body):
    ipa = ""
    if 'sounds' in body:
      
        for j in body['sounds']:
            if ('ipa' in j):
                ipa = j['ipa']
                
                
    return ipa







    

def formatObject(body):
    return {
        'pos'  : body['pos'],
        'etymology_text' : body['etymology_text'] if 'etymology_text' in body else "",
        'etymology_number' : body['etymology_number'] if 'etymology_number' in body else "",
        'definitions': formatUsage(body),
        'audio':formatAudio(body),
        'ipa':formatIPA(body),
        'source':'Wiktionary'
        

    }
    

# Getting all objects/entries related to the word in payload and associating them to word using above function

In [ ]:
# Getting all objects/entries related to the word in payload
payload= {}
count = 0
for i in lst[:]:
    word = i['word']
    if word in allWords:
        payload[word]= [formatObject(i)] if word not in payload else payload[word]+[formatObject(i)]
        

    

# This block will build the an array of json which can be loaded to a non sql database directly

In [ ]:
finalProcessedWords = []
for i in payload:
    finalProcessedWords.append({
        'word':i,
        'usage': payload[i],
        'type': 'word_data'
    })




# For Dumping the Araay to a json file

In [ ]:
json_object = json.dumps(finalProcessedWords, indent=4)
with open("sample.json", "w") as outfile:
    outfile.write(json_object)        


<a href="/kaggle/working/sample.json"> Download File </a>
